In [ ]:
%pip install pandas
%pip install transformers
%pip install accelerate

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [76]:
import pandas as pd
df = pd.read_csv('test_subset.csv')


In [ ]:
#text set for getting the results from LLM
texts = list(df.text)

### Chat propmt used to get the labels of sequences

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-1.1-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

prompt = f"Classify the sentiment of each text into ONE of the three classes: neutral, negative or positive. Split the answer in a dictionary with two keys:Text and Label. Text: "

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    print(answer)
print(len(answers))

### Pre-processing the results in the desired format , so as to evaluate the accuracy

In [ ]:
texts_after = []
labels_after = []
for answer in answers:
    if len((answer.split('<start_of_turn>model')[1].split(':'))) > 2:
        texts_after.append(answer.split('<start_of_turn>model')[1].split(':')[1].split('"Label"')[0].strip(' "" '))
        labels_after.append(answer.split('<start_of_turn>model')[1].split(':')[2].split('\n')[0].strip(' "" '))
    else:
        print('not proper',answer.split('<start_of_turn>model')[1].split(':'))
    
print(len(texts_after))
print(len(labels_after))

In [77]:
#dropping the row which was not preoperly formatted as seen from above results
df = df[(df['text'] != 'Was ist nur in Deutschland los, dass man ein furchtbares Unrecht nicht benennen darf ohne als Antisemit angeprangert zu werden? „Nie wieder“… hatten wir das nach dem 2. Weltkrieg nicht verkündet und seitdem in Sonntagsreden endlos wiederholt? Wie kann unsere Politik, können die Medien die Gräueltaten Israels rechtfertigen? Fragt man sich zudem nicht, wieso in Harvard und auch sonst in großen Teilen der Welt ein Ende der Bombardierung gefordert wird? Ach, ich frage mich übrigens, ob es um die Öl- und Gasvorkommen in Gaza geht 🙌')]
df = df.reset_index(drop=True)

In [ ]:

df['predicted_sentiment_llm'] = labels_after
#getting only the classes 'Negative','Positive,'and 'Neutral' and ignoring other classes
df = df[('Negative' == df['predicted_sentiment_llm']) | ('Positive' == df['predicted_sentiment_llm']) | ('Neutral' == df['predicted_sentiment_llm'])]
df = df.reset_index(drop=True)
#lower casing the class names
for i in range(len(df)):
    df.loc[i,'predicted_sentiment_llm'] = str(df['predicted_sentiment_llm'][i]).lower()

In [89]:
correct_predictions = 0
for i in range(len(df)):
    if df['sentiment'][i] == df['predicted_sentiment_llm'][i]:
        correct_predictions += 1
print('correct predictions:',correct_predictions)

correct predictions: 196


In [92]:
Accuracy = (correct_predictions/len(df))*100
print('Accuracy of google gemma before fine tuning is',Accuracy, '%')

Accuracy of google gemma before fine tuning is 76.26459143968872 %


In [93]:
df.to_csv('ResultsBeforeFineTuning_SequnceClassification_gemmaGoogle.csv', index=False)